In [89]:
import time
import json

import pandas as pd
import numpy as np
import seaborn as sns

import pyperclip

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

In [108]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# selenium 기본 설정
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

with open('settings.json') as f:
    settings = json.load(f)
id = settings['id']
password = settings['password']


# 웹페이지 해당 주소 이동
url = settings['url']
driver.get(url)
time.sleep(0.3)
frame = driver.find_element(By.XPATH, '/html/frameset/frame[2]')
driver.switch_to.frame(frame)
# 아이디 입력
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divMain_edtId_input"]').send_keys(id)
time.sleep(0.3)
# 비밀번호 입력
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divMain_edtPw_input"]').click()
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divMain_edtPw_input"]').send_keys(password)
time.sleep(0.3)
# 로그인 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divMain_btnLogin"]/div').click()


# 로그인 성공
# frame = driver.find_element(By.XPATH, '/html/frameset/frame[2]')
# driver.switch_to.frame(frame)
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divTop_img_TA_top_menu2"]/div').click()
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divLeftMenu_divLeftMainList_grdLeft_body_gridrow_5_cell_5_0_controltreeTextBoxElement"]/div').click()
time.sleep(0.3)
driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_form_divLeftMenu_divLeftMainList_grdLeft_body_gridrow_6_cell_6_0_controltreeTextBoxElement"]/div').click()
time.sleep(0.3)

In [109]:
GRID_MAX = 42

transaction_index = 0
data_list = []
while True:
    try:
        # 거래일자 추출
        date = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_3GridCellTextContainerElement"]').text
        time.sleep(0.05)

        # 거래시간 추출
        transaction_time = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_4GridCellTextContainerElement"]').text
        time.sleep(0.05)

        # 카드번호 추출
        card_number = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_7GridCellTextContainerElement"]').text
        time.sleep(0.05)

        # 카드사명 추출
        card_name = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_8GridCellTextContainerElement"]').text
        time.sleep(0.05)

        # 매입사명 추출
        purchase_name = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_9GridCellTextContainerElement"]').text
        time.sleep(0.05)

        # 영수증 버튼 클릭
        driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_form_divMain_divWork_grdCardSaleList_body_gridrow_{transaction_index}_cell_{transaction_index}_13_controlbuttonTextBoxElement"]/div').click()
        time.sleep(0.3)
    except NoSuchElementException:
        print('모든 영수증을 저장했습니다.')
        break
    except ElementNotInteractableException:
        print('모든 영수증을 저장했습니다.')
        break
    
    transaction_data = {
        'date': date,
        'time': transaction_time,
        'card_number': card_number,
        'card_name': card_name,
        'purchase_name': purchase_name,
        'items': []  # List to store item data for this transaction
    }

    item_index = 0
    while True:
        try:
        # 상품명 추출
            name = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_popupBill_form_grdDetailList_body_gridrow_{item_index}_cell_{item_index}_0GridCellTextContainerElement"]/div').text
            time.sleep(0.05)
            # 상품 가격 추출
            price = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_popupBill_form_grdDetailList_body_gridrow_{item_index}_cell_{item_index}_1GridCellTextContainerElement"]/div').text
            time.sleep(0.05)
            # 상품 수량 추출
            count = driver.find_element(By.XPATH, f'//*[@id="mainframe_childframe_popupBill_form_grdDetailList_body_gridrow_{item_index}_cell_{item_index}_2GridCellTextSimpleContainerElement"]').text
            time.sleep(0.05)
            item_data = {
                'name': name,
                'price': price,
                'count': count
            }
            transaction_data['items'].append(item_data)
            item_index += 1
        except NoSuchElementException:
            print('모든 아이템을 저장했습니다.')
            break
    data_list.append(transaction_data)
    # 영수증 화면 종료
    driver.find_element(By.XPATH, '//*[@id="mainframe_childframe_popupBill_titlebar_closebuttonAlignImageElement"]').click()
    time.sleep(0.3)
    transaction_index += 1
    transaction_index %= GRID_MAX

df = pd.DataFrame(columns=['date', 'time', 'card_number', 'card_name', 'purchase_name', 'name', 'price', 'count'])

for transaction_data in data_list:
    for item_data in transaction_data['items']:
        new_data = pd.DataFrame.from_dict([{
            'date': transaction_data['date'],
            'time': transaction_data['time'],
            'card_number': transaction_data['card_number'],
            'card_name': transaction_data['card_name'],
            'purchase_name': transaction_data['purchase_name'],
            'name': item_data['name'],
            'price': item_data['price'],
            'count': item_data['count']
        }])
        df = pd.concat([df, new_data])
        
df_0908 = df[:]
df_0908.to_csv('./datas/df_0908.csv', index=False)

모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 아이템을 저장했습니다.
모든 영수증을 저장했습니다.
